In [ ]:
import pandas as pd

In [ ]:
%run ../modules/utils.ipynb
%run ../modules/cds.ipynb
%run ../modules/preprocessing.ipynb

%run ../modules/paper_figures.ipynb

In [ ]:
glu_path = '/Users/xyuan/Documents/ChenLab/10Jun2021/degraded/DrChen_tRNA_A_Glu_FA_17.xlsx'
df_glu1 = load_data(glu_path)
glu_path = '/Users/xyuan/Documents/ChenLab/10Jun2021/degraded/18_Glu_A.xlsx'
df_glu1_alkb = load_data(glu_path)

# glu_path = '/Users/xyuan/Documents/ChenLab/8May2021/UC_glu_5_rosa.xlsx'
glu_path = '/Users/xyuan/Documents/ChenLab/DataYue/Glu/DrChen_tRNAs5_complete.xlsx'
df_glu2 = load_data(glu_path)
# glu_path = '/Users/xyuan/Documents/ChenLab/8May2021/UC_glu_6_rosa.xlsx'
glu_path = '/Users/xyuan/Documents/ChenLab/DataYue/Glu/DrChen_tRNAs6_complete.xlsx'
df_glu2_alkb = load_data(glu_path)

In [ ]:
# df_glu1 = df_glu1[df_glu1.Mass>1500]
# df_glu1_alkb = df_glu1_alkb[df_glu1_alkb.Mass>1500]
df_glu2 = df_glu2[df_glu2.Mass>1500]
df_glu2_alkb = df_glu2_alkb[df_glu2_alkb.Mass>1500]

In [ ]:
df_glu1.shape, df_glu1_alkb.shape, df_glu2.shape, df_glu2_alkb.shape

In [ ]:
df_wild_24k = df_glu1_without_adducts[df_glu1_without_adducts.Mass>24000]
df_alkb_24k = df_glu1_alkb_without_adducts[df_glu1_alkb_without_adducts.Mass>24000]
# df_wild_24k = df_glu2[df_glu2.Mass>24000]
# df_alkb_24k = df_glu2_alkb[df_glu2_alkb.Mass>24000]
plotly_zone(df_wild_24k, y='Vol')
plotly_zone(df_alkb_24k, y='Vol')
# plotly_zones(df_wild_24k[df_wild_24k.Vol>5E5], df_alkb_24k[df_alkb_24k.Vol>5E5], y='Vol')

In [ ]:
# remove adducts from df_gln2
adducts = [K, Na]
dfm_adducts_list = list()
for adduct in adducts:
    dfm_r = match_dfs(df_glu1, df_glu1, shift=adduct)
    dfm_adducts_list.append(dfm_r)
dfm_adducts = pd.concat(dfm_adducts_list).drop_duplicates()
df_glu1_without_adducts = df_glu1.drop(dfm_adducts.index)

dfm_adducts_list = list()
for adduct in adducts:
    dfm_r = match_dfs(df_glu1_alkb, df_glu1_alkb, shift=adduct)
    dfm_adducts_list.append(dfm_r)
dfm_adducts = pd.concat(dfm_adducts_list).drop_duplicates()
df_glu1_alkb_without_adducts = df_glu1_alkb.drop(dfm_adducts.index)

In [ ]:
# remove adducts from df_gln2
adducts = [K, Na]
dfm_adducts_list = list()
for adduct in adducts:
    dfm_r = match_dfs(df_glu2, df_glu2, shift=adduct)
    dfm_adducts_list.append(dfm_r)
dfm_adducts = pd.concat(dfm_adducts_list).drop_duplicates()
df_glu2_without_adducts = df_glu2.drop(dfm_adducts.index)

dfm_adducts_list = list()
for adduct in adducts:
    dfm_r = match_dfs(df_glu2_alkb, df_glu2_alkb, shift=adduct)
    dfm_adducts_list.append(dfm_r)
dfm_adducts = pd.concat(dfm_adducts_list).drop_duplicates()
df_glu2_alkb_without_adducts = df_glu2_alkb.drop(dfm_adducts.index)

In [ ]:
# df_wild_sample = df_glu2_without_adducts[df_glu2_without_adducts.Vol>df_glu2_without_adducts.Vol.quantile(.75)].copy()
# df_alkb_sample = df_glu2_alkb_without_adducts[df_glu2_alkb_without_adducts.Vol>df_glu2_alkb_without_adducts.Vol.quantile(.75)].copy()
# df_wild_sample.shape, df_alkb_sample.shape

In [ ]:
df_wild_sample = local_top(df_glu1_without_adducts, top=10)
df_alkb_sample = local_top(df_glu1_alkb_without_adducts, top=10)
# df_wild_sample = local_top(df_glu2_without_adducts, top=10)
# df_alkb_sample = local_top(df_glu2_alkb_without_adducts, top=10)
df_wild_sample.shape, df_alkb_sample.shape

In [ ]:
df_wild_sample_only, df_alkb_sample_only = diff_dfs(df_wild_sample, df_alkb_sample)
df_wild_sample_only.shape, df_alkb_sample_only.shape
# dfm_tmp = match_dfs(df_wild_sample, df_alkb_sample_only)
# dfm_tmp.shape


In [ ]:
import multiprocessing
from collections import namedtuple
ShiftHit = namedtuple('ShiftHit', 'shift hit')

def func(df_s1, df_s2, shift):
    dfm = match_dfs(df_s1, df_s2, shift=shift)
    sh = ShiftHit(shift=shift, hit=dfm.shape[0])
    return sh

PROCESSES = 8
params = [(df_wild_sample, df_alkb_sample_only, shift) for shift in np.arange(-64, 64, 0.1)]
# params = [(df_wild_sample, df_wild_sample, shift) for shift in np.arange(-64, 64, 1)]
# params = [(df_alkb_sample, df_alkb_sample, shift) for shift in np.arange(-64, 64, 1)]
# params = [(df_glu2, df_glu2_alkb_only, shift) for shift in np.arange(-64, 64, 1)]
# params = [(df_glu1, df_glu1_alkb, shift) for shift in np.arange(-64, 64, 1)]
# params = [(df_gln2, df_gln2, shift) for shift in np.arange(1, 64, 1)]
# params = [(df_gln2_without_adducts, df_gln2_alkb_without_adducts, shift) for shift in np.arange(-64, 64, 1)]
# params = [(df_comm_wild2, df_comm_wild2, shift) for shift in np.arange(1, 64, 1)]
# params = [(df_wild2, df_alkb2, shift) for shift in np.arange(-64, 64, 1)]
# params = [(dfm_l, dfm_l, shift) for shift in np.arange(1, 64, 1)]
# params = [(df_sample, shift) for shift in np.arange(1, 64, 0.1)]
with multiprocessing.Pool(PROCESSES) as pool:
    shift_hits = pool.starmap(func, params)

df_shift_hits = pd.DataFrame(shift_hits)

In [ ]:
fig = px.scatter(df_shift_hits[df_shift_hits.hit<df_shift_hits.shape[0]], x='shift', y='hit', labels={'shift': 'Mass Shift', 'hit': 'Hit Count'})
fig.show()

In [ ]:
# df_sh_glu2 = df_shift_hits.copy()
# df_sh_glu2.to_excel('ShiftHit_Glu2_top10.xlsx')

In [ ]:
global_mean = df_shift_hits.hit.mean()
df_calc = df_shift_hits[(df_shift_hits.hit<df_shift_hits.shape[0])&
                        (df_shift_hits.hit>global_mean)]
global_mean, df_calc.hit.mean()
# df_calc.hit.quantile(.75)

In [ ]:
# df_sh_full = df_shift_hits.copy()
# df_sh_alkb_only = df_shift_hits.copy()

In [ ]:
def func_count_basecallings(shift, df_s1, df_s2):
    dfm_l, dfm_r = peer_dfs(df_s1, df_s2, shift=shift)
    df = dfm_l if dfm_l.shape[0] < dfm_r.shape[0] else dfm_r
    bcr = base_calling_random(df)
    return len(bcr[1])

def func_longest_ladder(shift, df_s1, df_s2):
    dfm_l, dfm_r = peer_dfs(df_s1, df_s2, shift=shift)
    df = dfm_l if dfm_l.shape[0] < dfm_r.shape[0] else dfm_r
    groups = basecalling_groups(df)
    return len(groups[0][1]) if groups else 0

df_s1 = df_wild_sample.copy()
df_s2 = df_alkb_sample.copy()
df_sh_top = df_shift_hits.sort_values('hit', ascending=False).iloc[0:10].copy()
df_sh_top['basecallings'] = df_sh_top.apply(
    lambda x: func_count_basecallings(x['shift'], df_s1, df_s2), axis=1)
df_sh_top['longestladder'] = df_sh_top.apply(
    lambda x: func_longest_ladder(x['shift'], df_s1, df_s2), axis=1)
df_sh_top

In [ ]:
def func_longest_ladder(shift, df_s1, df_s2):
    dfm_l, dfm_r = peer_dfs(df_s1, df_s2, shift=shift)
    df = dfm_l if dfm_l.shape[0] < dfm_r.shape[0] else dfm_r
    groups_l = basecalling_groups(df)
    return len(groups_l[0][1])

func_longest_ladder(0, df_wild_sample, df_alkb_sample)

In [ ]:
fig = plt.figure(figsize=(10, 6))
df_plt = df_sh_top.copy().sort_values('shift')
df_plt1 = df_plt.copy()
df_plt1['shift'] = df_plt1['shift'].astype(int)
df_plt1 = df_plt1.set_index('shift')
df_plt1.plot(kind='bar')

In [ ]:
shift = -14.1
# dfm_l, dfm_r = peer_dfs(df_glu1, df_glu1_alkb, shift=shift)
dfm_base_l, dfm_base_r = peer_dfs(df_wild_sample, df_alkb_sample, shift=0)
dfm_l, dfm_r = peer_dfs(df_wild_sample, df_alkb_sample, shift=shift)
# dfm_l.shape, dfm_r.shape
plotly_zones(dfm_base_l, dfm_l)
bcr = base_calling_random(dfm_l)
plotly_basecalling(dfm_l, bcr[1])
bcr = base_calling_random(dfm_r)
plotly_basecalling(dfm_r, bcr[1], y='RT')

In [ ]:
dft_l = dfm_l[(dfm_l.Mass>3000)&(dfm_l.Mass<7700)]
bcr = base_calling_random(dft_l)
plotly_basecalling(*bcr)

dft_r = dfm_r[(dfm_r.Mass>2900)&(dfm_r.Mass<7700)]
bcr = base_calling_random(dft_r)
plotly_basecalling(*bcr)

In [ ]:
dftmp = pd.DataFrame({'Mass': [6070.89, 6415.932, 6720.984,
                              6108.838, 6453.874, 6758.932,
                               6056.877, 6401.92, 6706.967,
                              6094.815, 6439.872, 6744.916]})

df_ladders = match_dfs(dftmp, dft_l)
bcr_l = base_calling_random(df_ladders)
plotly_basecalling(*bcr_l)

dftmp.Mass -= M
df_ladders = match_dfs(dftmp, dft_r)
bcr_r = base_calling_random(df_ladders)
plotly_basecalling(*bcr_r)

In [ ]:
thres = 10000
groups_l = basecalling_groups(dfm_l[dfm_l.Mass<thres])
len(groups_l)
groups_r = basecalling_groups(dfm_r[dfm_r.Mass<thres])
len(groups_r)

In [ ]:
idx = 0

dft = groups_l[idx][0]
print(dft.shape)
bcr = base_calling_random(dft)
plotly_basecalling(*bcr)

dft = groups_r[idx][0]
bcr = base_calling_random(dft)
plotly_basecalling(*bcr)

In [ ]:
# df_sh_top = df_shift_hits.sort_values('hit', ascending=False).iloc[1:11].copy()
df_sh_top = df_shift_hits.sort_values('hit', ascending=False).iloc[0:10].copy()
# df_sh_top = df_sh_alkb_only.sort_values('hit', ascending=False).iloc[:10].copy()
df_sh_top

In [ ]:
fig = plt.figure(figsize=(2.5, 1.5))

# plot_basecalling(*bcr, figsize=(5, 3))
# bcr = base_calling_random(df_ladders)
nodes = pd.concat([bcr_l[0], bcr_r[0]])
edges = bcr_l[1] + bcr_r[1]
# plot_basecalling(*bcr_l, figsize=(5, 3))
# plot_basecalling(*bcr_r, figsize=(5, 3))
plot_basecalling(nodes, edges, figsize=(5, 3))

# plt.legend()
fig.tight_layout()
svg_fpath = '/Users/xyuan/Downloads/demo.svg'
plt.savefig(svg_fpath, transparent=True, dpi=300)

In [ ]:
def func(shift, df_s1, df_s2):
    dfm_l, dfm_r = peer_dfs(df_s1, df_s2, shift=shift)
    df = dfm_l if dfm_l.shape[0] < dfm_r.shape[0] else dfm_r
    bcr = base_calling_random(df)
    return len(bcr[1])

df_s1 = df_wild_sample.copy()
df_s2 = df_alkb_sample_only.copy()
# df_s1 = df_glu2.copy()
# df_s2 = df_glu2_alkb_only.copy()
df_sh_top['basecallings'] = df_sh_top.apply(lambda x: func(x['shift'], df_s1, df_s2), axis=1)
df_sh_top

In [ ]:
shifts = df_sh_top.iloc[:3]['shift'] #[-16, -15, -1, 1, 15]
print(shifts)
dfm_list = list()
for shift in shifts:
    dfm_r = match_dfs(df_wild_sample, df_alkb_sample_only, shift=shift)
    dfm_list.append(dfm_r)
# dfm_list.append(df_mock)
dfm_shifts = pd.concat(dfm_list).drop_duplicates()

In [ ]:
# plotly_zone(dfm_shifts)
dft = dfm_shifts
# dft = dfm_list[2]
bcr = base_calling_random(dft)
plotly_basecalling(*bcr)

In [ ]:
fig = plt.figure(figsize=(2.5, 1.5))
df_plt = df_sh_top.copy().sort_values('shift')
df_plt1 = df_plt.copy()
df_plt1['shift'] = df_plt1['shift'].astype(int)
df_plt1 = df_plt1.set_index('shift')
df_plt1.plot(kind='bar',figsize=(2.5, 1.5))
plt.legend()

fig.tight_layout()
svg_fpath = '/Users/xyuan/Downloads/demo.svg'
plt.savefig(svg_fpath, transparent=True, dpi=300)

In [ ]:
svg_fpath = '/Users/xyuan/Downloads/demo.svg'
draw_df(df_sh_alkb_only[df_sh_alkb_only.hit>0], svg_fpath, x='shift', y='hit')